In [64]:
# импортируем библиотеки
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from imblearn.under_sampling import RandomUnderSampler
%matplotlib inline

In [14]:
Admission_train = pd.read_csv('Admission_train.csv')
Admission_test = pd.read_csv('Admission_test.csv')

In [15]:
Admission_train.drop(['Unnamed: 0', 'Serial No.'], axis = 1, inplace=True)
Admission_test.drop(['Unnamed: 0', 'Serial No.'], axis = 1, inplace=True)

In [16]:
Admission_train['Chance of Admit '] = Admission_train['Chance of Admit '].apply(lambda x: 1 if x >= 0.5 else 0)
Admission_test['Chance of Admit '] = Admission_test['Chance of Admit '].apply(lambda x: 1 if x >= 0.5 else 0)

In [22]:
Adm_train_cel_per = Admission_train['Chance of Admit ']
Adm_train_per = Admission_train.drop(['Chance of Admit '], axis = 1)
Adm_test_cel_per = Admission_test['Chance of Admit ']
Adm_test_per = Admission_test.drop(['Chance of Admit '], axis=1)

In [30]:
# чтобы не делать 100500 копипастов, создадим функцию print_logit_scores
# которая будет обучать регрессию заданным способом и выводить разные метрики качества

def print_logit_scores(Adm_train_per, Adm_train_cel_per, Adm_test_per, Adm_test_cel_per, model_type, weights):
    
    # data_train, target_train, data_test, target_test - это обучающие и тестовые данные
    # model_type задает один из 3 типов обучения: 'n' - None, 'b' - balanced, 'w' - заданные пользователем веса
    # w - вектор весов. Используется только для model_type = 'w'
    
    if (model_type == 'n'): # обучаем с равными весами
        lm = linear_model.LogisticRegression(solver='liblinear', class_weight=None)    
    elif (model_type == 'b'): # балансируем веса, как предлагают разработчики sklearn
        lm = linear_model.LogisticRegression(solver='liblinear', class_weight='balanced')
    elif (model_type == 'w'): # балансируем веса самостоятельно
        lm = linear_model.LogisticRegression(solver='liblinear', class_weight={0:weights[0], 1:weights[1]}) 

    # обучаем
    model = lm.fit(Adm_train_per, Adm_train_cel_per.values.ravel()) 

    # сделаем prediction классов на всей тестовой выборке
    target_pred = lm.predict(Adm_test_per)

    # строим confusion matrix - таблицу правильных и неправильных предсказаний
    cnf_matrix = metrics.confusion_matrix(Adm_test_cel_per, target_pred)

    TN = cnf_matrix[0,0] # True Negative
    TP = cnf_matrix[1,1] # True Positive
    FN = cnf_matrix[1,0] # False Negative
    FP = cnf_matrix[0,1] # False Positive
    
    Ac = lm.score(Adm_test_per, Adm_test_cel_per)
    Sens = TP/(TP+FN) 
    Sp = TN/(TN+FP)
    P = TP/(TP+FP)
    typeI = FP/(FP+TN)
    typeII = FN/(FN+TP)
    
    print('Accuracy: ', Ac)
    print('Sensitivity: ', Sens)
    print('Specificity: ', Sp)
    print('Precision: ', P)
    print('Type I error rate: ', typeI)
    print('Type II error rate: ', typeII)
    
    return [Ac,Sens,Sp,P,typeI,typeII] # возвращаем список метрик

In [24]:
share = Adm_test_cel_per.value_counts()
w0 = share[1]/(share[0]+share[1])
w = np.array([w0,1-w0])
w

array([0.91, 0.09])

In [27]:
np.bincount(Adm_test_cel_per) # считает количество вхождений 0 и 1 в y_train['TenYearCHD']
w_b = Adm_test_per.shape[0]/ (2*np.bincount(Adm_test_cel_per))
w_b

array([5.55555556, 0.54945055])

In [28]:
# Давайте убедимся, что отношения весов действительно одинаковые
# При этом бОльший по размеру класс (нулевой, то есть здоровые пациенты) имеет мЕньший вес
print('отношение интуитивных весов: ', w[0]/w[1])
print('отношение balanced весов: ', w_b[0]/w_b[1])

отношение интуитивных весов:  10.111111111111114
отношение balanced весов:  10.11111111111111


In [31]:
# сравним 
print('ручная балансировка по правилу balanced')
m1 = print_logit_scores(Adm_train_per, Adm_train_cel_per, Adm_test_per, Adm_test_cel_per, 'w', w_b) # ручная балансировка по правилу balanced
print('\n')
print ('встроенная балансировка по правилу balanced')
m2 = print_logit_scores(Adm_train_per, Adm_train_cel_per, Adm_test_per, Adm_test_cel_per, 'b', w) # встроенная балансировка по правилу balanced
print('\n')
print ('без балансировки весов')
m3 = print_logit_scores(Adm_train_per, Adm_train_cel_per, Adm_test_per, Adm_test_cel_per, 'n', w) # без балансировки весов

ручная балансировка по правилу balanced
Accuracy:  0.78
Sensitivity:  0.7802197802197802
Specificity:  0.7777777777777778
Precision:  0.9726027397260274
Type I error rate:  0.2222222222222222
Type II error rate:  0.21978021978021978


встроенная балансировка по правилу balanced
Accuracy:  0.78
Sensitivity:  0.7802197802197802
Specificity:  0.7777777777777778
Precision:  0.9726027397260274
Type I error rate:  0.2222222222222222
Type II error rate:  0.21978021978021978


без балансировки весов
Accuracy:  0.91
Sensitivity:  0.967032967032967
Specificity:  0.3333333333333333
Precision:  0.9361702127659575
Type I error rate:  0.6666666666666666
Type II error rate:  0.03296703296703297


In [75]:
# освежите в памяти, что показывает share и что значит share[1]
# параметр ratio в RandomUnderSampler задается словарем: 
# 1: - желаемое количество объектов класса 1
# 0: - желаемое количество объектов класса 0

# задаем параметры выборки:
sampler = RandomUnderSampler(sampling_strategy = {1: share[0], 0: share[0]})

# сам unpersampling выполняется здесь:
X_train_under_np, y_train_under_np = sampler.fit_sample(Adm_train_per.values, Adm_train_cel_per.values)

# преобразуем в DataFrame, чтобы скормить логистической регрессии
X_train_under = pd.DataFrame(X_train_under_np)
y_train_under = pd.DataFrame(y_train_under_np)

In [80]:
# вычисляем качество модели с undersampling
m_u = print_logit_scores(X_train_under, y_train_under, Adm_test_per, Adm_test_cel_per, 'n', w)

Accuracy:  0.77
Sensitivity:  0.7692307692307693
Specificity:  0.7777777777777778
Precision:  0.9722222222222222
Type I error rate:  0.2222222222222222
Type II error rate:  0.23076923076923078


In [79]:
# сравните результаты со встроенной балансировкой на всей обучающей выборке
# как вы думаете, какой есть существенный недостаток у undersampling по сравнению с балансировкой весов?
m2 = print_logit_scores(Adm_train_per, Adm_train_cel_per, Adm_test_per, Adm_test_cel_per, 'b', w)

Accuracy:  0.78
Sensitivity:  0.7802197802197802
Specificity:  0.7777777777777778
Precision:  0.9726027397260274
Type I error rate:  0.2222222222222222
Type II error rate:  0.21978021978021978
